In [1]:
# Test if gpu-support is working.
import xgboost as xgb
from sklearn.datasets import load_boston

boston = load_boston()

# XGBoost API example
params = {'tree_method': 'gpu_hist', 'max_depth': 3, 'learning_rate': 0.1}
dtrain = xgb.DMatrix(boston.data, boston.target)
xgb.train(params, dtrain, evals=[(dtrain, "train")])

# sklearn API example
gbm = xgb.XGBRegressor(silent=False, n_estimators=10, tree_method='gpu_hist')
gbm.fit(boston.data, boston.target, eval_set=[(boston.data, boston.target)])

[0]	train-rmse:21.60208
[1]	train-rmse:19.55555
[2]	train-rmse:17.71453
[3]	train-rmse:16.06071
[4]	train-rmse:14.57054
[5]	train-rmse:13.23501
[6]	train-rmse:12.03721
[7]	train-rmse:10.94916
[8]	train-rmse:9.98378
[9]	train-rmse:9.10604
[15:48:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:17.06433
[1]	validation_0-rmse:12.27140
[2]	validation_0-rmse:8.92523
[3]	validation_0-rmse:6.56554
[4]	validation_0-rmse:4.88678
[5]	validation_0-rmse:3.70939
[6]	validation_0-rmse:2.89225
[7]	validation_0-rmse:2.33943
[8]	validation_0-rmse:1.94412
[9]	validation_0-rmse:1.66867


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
             subsample=1, tree_method='gpu_hist', validate_parameters=1,
             verbosity=None)

In [103]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(' old_training_data\session_buy_data_2_14.csv')
test = pd.read_csv(' old_training_data\session_buy_data_1.csv')
buy=data.groupby('buy').get_group(1)
nobuy=data.groupby('buy').get_group(0)
buy.info()
nobuy.info()
train = pd.concat([buy,nobuy.head(100000)])
x_train = train[['item_id','clicks']]
y_train = train['buy']
x_test = test[['item_id','clicks']]
y_test = test['buy']
#x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state = 33)
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1,tree_method='gpu_hist')
xgbc.fit(x_train, y_train)
print("Accu:",xgbc.score(x_test,y_test))
test['pred'] = xgbc.predict(x_test)
print(test)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106133 entries, 23 to 2705416
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   session  106133 non-null  int64
 1   item_id  106133 non-null  int64
 2   clicks   106133 non-null  int64
 3   buy      106133 non-null  int64
dtypes: int64(4)
memory usage: 4.0 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2599291 entries, 0 to 2705423
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int64
 1   item_id  int64
 2   clicks   int64
 3   buy      int64
dtypes: int64(4)
memory usage: 99.2 MB
[21:14:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue 

c:\python39\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Accu: 0.5185393163649684
         session    item_id  clicks  buy  pred
0              1  214536500       1    0     0
1              1  214536506       1    0     0
2              1  214577561       1    0     0
3              1  214536502       1    0     0
4            128  214826615       1    0     0
...          ...        ...     ...  ...   ...
414924  11561984  214712242       1    0     0
414925  11562112  214651419       1    0     0
414926  11562112  214829034       1    0     1
414927  11562113  214658096       1    0     0
414928  11562113  214855046       1    0     1

[414929 rows x 5 columns]


In [2]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.DataFrame()
for i in range(0,29):
    rpath = "train_data\\train_data_p"+str(i)+".csv"
    print("[TRAIN] Reading:",rpath)
    d0 = pd.read_csv(rpath)
    data = pd.concat([data,d0])

test = pd.read_csv('train_data\\train_data_p29.csv')

x_train = data[['sales','duration','items','clicks']]
y_train = data['buy']
x_test = test[['sales','duration','items','clicks']]
y_test = test['buy']

print("Trainning data size:",x_train.shape)
print("Testing data size:",x_test.shape)

buy = data.groupby('buy')
ybuy = buy.get_group(1)
nbuy = buy.get_group(0).head(115000)
ybuy.info()
nbuy.info()

[TRAIN] Reading: train_data\train_data_p0.csv
[TRAIN] Reading: train_data\train_data_p1.csv
[TRAIN] Reading: train_data\train_data_p2.csv
[TRAIN] Reading: train_data\train_data_p3.csv
[TRAIN] Reading: train_data\train_data_p4.csv
[TRAIN] Reading: train_data\train_data_p5.csv
[TRAIN] Reading: train_data\train_data_p6.csv
[TRAIN] Reading: train_data\train_data_p7.csv
[TRAIN] Reading: train_data\train_data_p8.csv
[TRAIN] Reading: train_data\train_data_p9.csv
[TRAIN] Reading: train_data\train_data_p10.csv
[TRAIN] Reading: train_data\train_data_p11.csv
[TRAIN] Reading: train_data\train_data_p12.csv
[TRAIN] Reading: train_data\train_data_p13.csv
[TRAIN] Reading: train_data\train_data_p14.csv
[TRAIN] Reading: train_data\train_data_p15.csv
[TRAIN] Reading: train_data\train_data_p16.csv
[TRAIN] Reading: train_data\train_data_p17.csv


KeyboardInterrupt: 

In [156]:
train_new = pd.concat([ybuy,nbuy])
test = pd.read_csv('train_data_p9.csv').head(25000)

x_train = train_new[['sales','duration','items','clicks']]
y_train = train_new['buy']
x_test = test[['sales','duration','items','clicks']]
y_test = test['buy']

print("Trainning data size:",x_train.shape)
print("Testing data size:",x_test.shape)

xgbc = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=9,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 scale_pos_weight=1,
 seed=27,
 tree_method='gpu_hist')
xgbc.fit(x_train, y_train)
print("Accuracy:",xgbc.score(x_test,y_test))
test['pre']=xgbc.predict(x_test)
print("Actually buys:",test.groupby('buy').get_group(1))
print("Predict buys:",test.groupby('pre').get_group(1))

Trainning data size: (230125, 4)
Testing data size: (15000, 4)
[15:27:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.7075333333333333
Actually buys:        session  sales  duration  clicks  items  buy  pre
10        1673      0   177.606       6      4    1    1
30        4873      0   824.941       5      3    1    1
36        5769      0    19.296       2      1    1    1
39        6281      0  1038.141       2      1    1    1
40        6409      0  3666.847       9      3    1    1
...        ...    ...       ...     ...    ...  ...  ...
14838  2374153      0   103.209       3      3    1    0
14850  2376073      0   420.472       5      4    1    1
14897  2383497      0    19.561       2      1    1    1
14907  238516

In [126]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional     scklearn functions
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV   #Perforing grid search

def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['buy'],eval_metric='auc')
    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['buy'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['buy'], dtrain_predprob))
                
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
    
target = 'buy'
IDcol = 'session'

predictors = [x for x in train.columns if x not in [target,IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27,
 tree_method='gpu_hist')

modelfit(xgb1, train, predictors)


Model Report
Accuracy : 0.799
AUC Score (Train): 0.882039


TypeError: 'NoneType' object is not callable